# Uncertainty and errors
If there needs to be one thing you must remember from this lesson, let it be this: every measurement or prediction in physics comes with an associated uncertainty. Always, no exception. That is because measurements provide an *estimate* of the true value, not the true value itself. 

In the context of quantitative measurements, the words **uncertainty** and **error** are sometimes used interchangeably. To be precise, error generally refers to the difference between a measured value and the true or accepted value, while uncertainty is an estimate of the range within which the true value is likely to lie, considering all known errors.

We cannot stress it enough: uncertainty is always present, whether we are reporting a measurement or making a prediction from the theory. Therefore it is important to know how to handle it quantitatively, regardless of where your interest in biophysics will take you.

> **Philosophical digression**
>
>But what about the fundamental constants on Nature, like for example the speed of light in vacuum? Do they come with an uncertainty too? In principle no, but only because they become [axioms](https://en.wikipedia.org/wiki/Axiom) of the physical framework used. For this we must be certain that they are *really* constant. In other words, whenever measured properly, the obtained value must always be within the range of uncertainty. In that case and that one only, it can be considered a fixed value of Nature and reported without error. The idea that, if we use better, more sensitive measuring tools, we would still get the same value, just with a smaller uncertainty.
>
>Not for nothing, the speed of light in vacuum is so constant it is used together with the definition of [second](https://en.wikipedia.org/wiki/Second) to define indirectly what the [meter](https://en.wikipedia.org/wiki/Metre) is. In the next chapter we will talk about units and dimensions, and we will come back at it.
>

### Types of errors
There are countless reasons why we cannot be certain about the values we measure. Consider for instance the measuring of the length of the table: maybe the length changes slightly from side to side due to imperfect manufacturing, or the meter we use cannot reach micrometric precision, or we accidentally measure between two points that are not exactly parallel to the length, or some evil crazy scientist replaced the meter with an inaccurate one, and so on. Some of this errors can be prevented, some other cannot. This does not mean that we cannot be precise in our scientific claims, but only that we acknowledge that what we know is limited to the tools we have to explore Nature. 

Errors are generally classified between these categories:
- **Random errors**: Variations that occur unpredictably from one measurement to another, mostly due to the nature of the process. They cannot be minimized at the source, but they can be reduced by averaging over a large number of observations. An example are the differences in protein concentrations expressed by the same cell line.
- **Systematic errors**: These are consistent, repeatable errors due to flaws in measurement instruments or procedures. Most systematic errors take the form of *biases*. They can and should be minimized at the source by properly calibrating the system. Unlike random errors, systematic errors cannot be reduced by increasing the number of observations. An example is not accounting for the mass of the weighing paper when measuring the weight of a powder.
- **Blunders**: These are careless human errors that happen randomly and result in (completely) off results. Compared to systematic errors, blunders are due to tiredness or negligence and generate biases hard to quantify. They can be minimized with good attention, careful practices and supervision of other colleagues who can point out the obvious mistakes. An example is pipetting $1\,mL$ of trypsin instead of $1\,\mu L$, or recording a $9$ instead of a $6$ onto a lab report.

Some textbooks do not distinguish between systematic errors and blunders; here I want to stress that, while systematic errors can in some cases be corrected in retrospective, blunders should be categorically excluded from analysis.
 
### Instrument resolution
Take a ruler and ask yourself: can I measure micrometers with this?

The answer is no. That is because the spacing between ticks on the ruler is $1\,mm$: finer ticks would be hard if not impossible to see with the naked eye, and due to [parallax](https://en.wikipedia.org/wiki/Parallax) they would literally be useless as different observers would report different measurements.

The instrument resolution (sometimes called *sensitivity*) represents the smallest increment an instrument can reliably measure, its "least count".

If we are measuring the length of a table, even after removing all other sources of errors, we cannot get an estimate of the true value more precise than the resolution of the measuring tool used. The only way to reduce this uncertainty is to use more sensitive tools with better resolution.

### Relative error 
explain

# Statistics of random errors

> **Note**
>
>In the following section you will find some interactive tools to help you visualize the concepts presented.
>To run the simulations you first need to click {fa}`rocket` --> {guilabel}`Live Code` on the top right corner of this page, and wait until the kernel has loaded. Then, you need to run the block of code by clicking  {guilabel}`run`.
>

When we say the outcome of a measurement is "random", we do not imply "unquantifiable". It is possible that the same object gives different outcomes when measured sequentially in time, or that within a group of objects the measured properties vary in a predetermined range, or both. What is the true value in these cases? What is the uncertainty? 

Consider for example the height distribution of various tree species.

In [ ]:
import numpy as np
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, CustomJS, Span, BoxAnnotation, Label
from bokeh.events import MouseMove

# For inline display in a notebook; if running as a script, consider using output_file("interactive.html")
output_notebook()

# Load the CSV file and clean column names
df = pd.read_csv("tree_datasets.csv")
df.columns = [col.strip() for col in df.columns if col.strip() != ""]

# Define coordinate pairs based on header names: (x, y)
pairs = [
    ("Ix", "Illicium verum"),
    ("Mx", "Masson pine"),
    ("Cx", "Chinese fir"),
    ("Sx", "Splash pine")
]

# Define the Okabe–Ito color palette (for 4 curves)
okabe_colors = ["#E69F00", "#56B4E9", "#009E73", "#F0E442"]

# Prepare lists for the renderers and annotation objects, and to store each curve's weighted mean and label
renderers = []
spans = []
left_bands = []
right_bands = []
labels = []  # for showing mean and std info
means = []   # weighted means for each curve

# Create a Bokeh figure and set axis labels
p = figure(title="Tree Height Distribution in Southern China", width=800, height=400, tools="")
p.xaxis.axis_label = "Height [m]"
p.yaxis.axis_label = "Relative frequency [a.u.]"

for i, (xcol, ycol) in enumerate(pairs):
    if xcol in df.columns and ycol in df.columns:
        color = okabe_colors[i]
        # Create a data source for the curve
        source = ColumnDataSource(data=dict(x=df[xcol], y=df[ycol]))
        # Plot the curve with the chosen color
        r = p.line('x', 'y', source=source, line_width=2, alpha=1.0, line_color=color, legend_label=f"{ycol}")
        renderers.append(r)
        
        # Compute the weighted mean and standard deviation (using y as frequency)
        weights = df[ycol]
        xs = df[xcol]
        mean_val = (xs * weights).sum() / weights.sum()
        means.append(mean_val)  # Save for later use in the callback
        std_val = np.sqrt((weights * (xs - mean_val)**2).sum() / weights.sum())
        
        # Interpolate to find the y-value at x = mean_val
        mean_y = np.interp(mean_val, xs, df[ycol])
        
        # Create a vertical dashed line at the weighted mean
        span = Span(location=mean_val, dimension='height', line_color=color,
                    line_dash='dashed', line_width=2, visible=False)
        # Create BoxAnnotations for ±1 standard deviation around the mean
        left_band = BoxAnnotation(left=mean_val - std_val, right=mean_val,
                                  fill_color=color, fill_alpha=0.2, visible=False)
        right_band = BoxAnnotation(left=mean_val, right=mean_val + std_val,
                                   fill_color=color, fill_alpha=0.2, visible=False)
        # Create a Label displaying the mean (as <h>) and std (as σ)
        label_text = "<h>: {:.2f}, σ: {:.2f}".format(mean_val, std_val)
        label = Label(x=mean_val, y=mean_y, x_offset=30, y_offset=5,
                      text=label_text, text_color=color, text_font_size="10pt", visible=False)
        
        # Add annotations and label to the plot
        p.add_layout(span)
        p.add_layout(left_band)
        p.add_layout(right_band)
        p.add_layout(label)
        
        spans.append(span)
        left_bands.append(left_band)
        right_bands.append(right_band)
        labels.append(label)
    else:
        print(f"Column pair ({xcol}, {ycol}) not found in DataFrame.")

# CustomJS callback: highlight the curve whose weighted mean is closest to the mouse's x position.
callback_code = """
let best_index = -1;
let best_dist = Infinity;
for (let i = 0; i < means.length; i++) {
    const dist = Math.abs(cb_obj.x - means[i]);
    if (dist < best_dist) {
        best_dist = dist;
        best_index = i;
    }
}

// Update each curve's opacity and the visibility of its annotations based on the best match.
for (let i = 0; i < renderers.length; i++) {
    if (i === best_index) {
        renderers[i].glyph.line_alpha = 1.0;
        spans[i].visible = true;
        left_bands[i].visible = true;
        right_bands[i].visible = true;
        labels[i].visible = true;
    } else {
        renderers[i].glyph.line_alpha = 0.25;
        spans[i].visible = false;
        left_bands[i].visible = false;
        right_bands[i].visible = false;
        labels[i].visible = false;
    }
}
p.change.emit();
"""

# Create and attach the callback; pass the renderers, annotations, labels, and means to the JS callback.
callback = CustomJS(args=dict(renderers=renderers, spans=spans, left_bands=left_bands, 
                              right_bands=right_bands, labels=labels, p=p, means=means), code=callback_code)
p.js_on_event(MouseMove, callback)

p.legend.location = "top_left"
p.legend.click_policy = "hide"

show(p)


ModuleNotFoundError: No module named 'bokeh'

<center>

*(Data from Wu, Y.; Zhang, X. Object-Based Tree Species Classification Using Airborne Hyperspectral Images and LiDAR Data. Forests 2020, 11, 32.)*

</center>

We can still describe some general properties of each species, the most obvious being the average height and the spread of the distribution. By hovering your mouse over the curves you will see the **mean** as a vertical dashed line and the **standard deviation** as a colored range around the mean (mean and standard deviation are the accurate statistical terms for average and spread of a distribution, but we'll get there in a minute).

You can notice a couple of things from this example:
- Nobody would dispute the claim "A *Masson pine* in southern China grows around 8.6 m tall". You should therefore not be surprised to see a 8.6 m tall *Masson pine*. The mean is then a good descriptor for the tree distribution;
- By looking at the highlighted range by the standard deviation you can see that most trees fall in that range. Similarly to the above claim, you should not be surprised to see a 10.3 m tall Masson pine;
- Without knowing absolutely anything about the phenotypes of these trees, if we see a tree 11.0 m tall, there is a very high chance that it is a *Chinese fir* and not a *Illicium verum*. More in general, we can safely claim that *Splash pines* grow taller than *Masson pines*.

We can say that the mean is the best estimator for the "true value" of the tree height, and the standard deviation the best estimator for the uncertainty. There you go, you have the recipe for describing statistical errors! But why? Are there better ways to describe statistical errors?

Think about how many genes could affect the height of a tree. Biology tells us that height is a complex, polygenic trait: it's not controlled by a single gene, but by the cumulative effect of many genes, each contributing a small amount to the final phenotype. So, even assuming that the chance of a mutation in an individual gene is completely unpredictable and all possible mutations have the same probability (like rolling a die), we end up with a smooth, continuous, bell-shaped distribution of heights. To understand why we need to look more quantitatively into the math.

### Estimation of the "true value" from sampled data
Mean, variance, standard deviation, corrected standard deviation, Standard error of the mean $m$

x=<x>+-std

### Discrete random processes
The most common example of a discrete random (stochastic) process is the roll of a die. While *Dungeons & Dragons* players might be familiar with many different types of dice, the most common is the cubic die, with 6 equally probable faces (1 to 6).

To describe this process mathematically, we define very generally the *probability $P$ of a measurement $X$ to have an outcome $x$*:

$$
P\big(X=x\big) = \frac{\text{number of favourable cases}}{\text{number of total cases}}
$$

So, for a regular cubic die (1d6), we have:

$$
P(1)=P(2)=...=P(6)=\frac{1}{6}
$$

What is the probability of getting a 1.5, 0 or a 7? Null! So we can write:

$$
P(0)=P(1.5)=P(7)=0
$$

Not very surprising, but technically correct.

One important yet trivial property is that the sum of all probabilities must be one:

$$
\Sigma P = 1
$$

In other words, the probability of getting a number between 1 and 6 is 100%. Again, not very surprising.

### From discrete to continuous: the central limit theorem
Now consider the distribution of the sum obtained by rolling two dice (2d6). The total number of different combinations is $6\cdot 6=36$.  The simplest way to describe the combined probability is to count all the combinations of possible outcomes:

<center>

|First die|Second die| Sum| 
| :-: | :-: | :-: |
| 1 | 1 | 2 |
| 1 | 2 | 3 | 
| 1 | 3 | 4 | 
| ... | ... | ... | 
| 6 | 4 | 10 |
| 6 | 5 | 11 |
| 6 | 6 | 12 | 

</center>

There is only one combination that gives sum 2 (1+1), so $P(2)=\frac{1}{36}$; similarly, there is only one combination that gives sum 12 (6+6), so $P(12)=\frac{1}{36}$ too. But then, there are many more combinations that give 7 (1+6, 2+5, 3+4, 4+3, 5+2, 6+1) so $P(7)=\frac{6}{36}$. Something interesting happens: combining perfect equal distributions of probability, we get a distribution that is not equal at all! This new distribution, for the mathematically inclined, is called **binomial distribution**.

Try to play now with the number of dice you sum and see how the probability distribution changes: 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

def slider_with_units(value, min_val, max_val, step, readout_format, description, unit):
    slider = widgets.FloatSlider(
        value=value, 
        min=min_val, 
        max=max_val, 
        step=step,
        readout=False,
        description=f"{description} = {value:{readout_format}} {unit}",
        style={'description_width': '100px'},
        layout=widgets.Layout(width='500px')
    )
    slider.observe(lambda change: slider.set_trait('description', 
                           f"{description} = {change['new']:{readout_format}} {unit} "), names='value')
    return slider

def plot_sum(n):
    n = int(n)
    # Compute the distribution of the sum of n dice by convolving the PMF of one die n times.
    # Each die has outcomes 1 through 6 with equal probability.
    p_die = np.ones(6) / 6
    p = p_die.copy()
    for _ in range(n - 1):
        p = np.convolve(p, p_die)
        
    # The possible sums range from n (all dice show 1) to 6*n (all dice show 6)
    sums = np.arange(n, 6*n + 1)
    
    plt.figure(figsize=(8, 4))
    # Plot black dots for each probability
    plt.plot(sums, p, 'ko')
    # Draw a thin vertical line from each dot to the x-axis
    for s, prob in zip(sums, p):
        plt.vlines(s, 0, prob, colors='k', linestyles='solid', linewidth=0.5)
    
    plt.xlabel("Sum")
    plt.ylabel("Probability")
    plt.xlim(n, 6*n)
    plt.ylim(0, max(p) * 1.1)
    # Remove grid, title, and legend as requested
    plt.show()

interact(plot_sum,
         n=slider_with_units(1, 1, 50, 1, '.0f', "n dice", ""))


Even from using only four dice we get a distribution that recalls a bell. The higher the number of dice, or stochastic factors in general, the smoother and more defined the distribution of outcomes will look like. And, in the limit of very large number of dice, the distribution will follow what is called a **Gaussian distribution**. This tendency of binomials to approximate a Gaussian curve is called **central limit theorem** and can be proven mathematically.

Wait, is this Gaussian distribution like the distribution of tree heights? Yes indeed. We mentioned that many (hundreds? thousands?) individual genes determine the height, and it makes perfect sense in a world where the phenotype is largely determined by the genotype. The Gaussian distribution is so common in Nature that is called the **"normal distribution"**. When a stochastic process is influenced by countless random factors, most probably its distribution will follow a Gaussian curve.

If your attention levels are still high, you might have noticed that the maximum probability decreases as the number of randomic factor increases. That is because, when rolling 50 dice, the probability of getting *exactly* 175 is very little: $P(35)=0.033$. That is because 174 and 176 are also quite probable. So, if we want to have a meaningful description of continuous stochastic processes, we need to slightly adjust the math accordingly.

### Continuous random processes
It is completely pointless to ask what is the probability of a tree being *exactly* 8 m tall. That's because if the height is infinitesimally different from 8, like 8.000000...0001 m, it would not count as a favorauble outcome. So if we calculate it in the continuous case, we would get $P(8)\approx 0$. What is more useful is to ask, instead, what is the probability of an outcome being between a certain range $x$ and $x+\Delta x$. 
If the $\Delta x$ is very small, we can write:

$$
\Delta x \to dx, \qquad P(x\le X \le x+dx) \to p(x)dx
$$

where $p(x)$ (notice the lowercase) is a continuous function describing the shape of the distribution, called **probability density**.

Then to get the total probability in a certain range one can sum all the infinitesimal probabilities:

$$
P(x\le X \le x+\Delta x)=\int_x^{x+\Delta x} p(x)dx
$$

Similarly to the discrete distributions, the normalization condition is:
$$
\int_{-\infty}^{+\infty} p(x)dx=1
$$



average, variance and std in continuous



Now that we have a framework for continuous distributions, we can say that the central limit theorem describes the probability density for a Gaussian stochastic process:

$$
p(x)=\frac{1}{\sqrt{2\pi\sigma^2}}e^{-\frac{(x-<x>)^2}{2\sigma^2}}
$$

where $<x>$ is the mean and $\sigma$ the standard deviation.



Estimates for $\mu$ and $\sigma$ from discrete measurements: mean, corrected sampled standard deviation, root mean squared error
Standard deviation of the mean

### Back to errors: which one to choose?
Say the sensitivity of an instrument is $s$. By using this instrument we collect many data points and we calculate the standard deviation $\sigma$. Which value to use as an uncertainty?

The short answer is: the largest value between $s$ and $\sigma$. That's because if the standard deviation is below the sensitivity, the individual variations may just be due to the imprecision of the instrument and not to a natural random variability. However, if the standard deviation is greater than the sensitivity, we know that the individual variations are all well measured with the instrument, despite its limited resolution.

# Error propagation
Recap: Taylor expansion
Taylor expansion of the velocity
General formula
Special case of power laws